In [192]:
# Import and load report
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_excel('report.xls')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828 entries, 0 to 827
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Deal        828 non-null    int64  
 1   Open time   828 non-null    object 
 2   Type        828 non-null    object 
 3   Size        828 non-null    float64
 4   Item        814 non-null    object 
 5   Price       828 non-null    float64
 6   S/L         828 non-null    int64  
 7   T/P         828 non-null    float64
 8   Time        828 non-null    object 
 9   Close       828 non-null    float64
 10  Commission  828 non-null    int64  
 11  Swap        828 non-null    float64
 12  Profit      828 non-null    float64
 13  Comment     828 non-null    object 
dtypes: float64(6), int64(3), object(5)
memory usage: 90.7+ KB


In [193]:
# Convert types and describe
CN = {'Open': 'Open time',
      'Close': 'Time',
      'Comment': 'Comment',
      'OrderID': 'Deal',
      'Profit': 'Profit',
      'Side': 'Type',
      'Qty': 'Size',
      
      'OpenPrice': 'Price',
      'TakeProfitPrice': 'T/P',

      'BalanceIn': 'DK_BALANCE_IN',
      'BalanceOut': 'DK_BALANCE_OUT',
      'WorstPrice': 'DK_WORST_PRICE',
      'OpenValue': 'DK_OPEN_VALUE',
      'NetID': 'DK_NET_ID',
      'NetQty': 'DK_NET_QTY',
      'NetValue': 'DK_NET_VALUE',
      'NetLastPrice': 'DK_NET_LAST_PRICE',
      'NetAvgPrice': 'DK_NET_AVG_PRICE',

      'Drawdown': 'DK_DRAWDOWN',
      'DrawdownRatio': 'DK_DRAWDOWN_RATIO',
      }

COMMENT_PATTERN_FOR_ORDERS_ONLY = "[tp]"
OPEN_NEW_ORDER_PRICE_DELTA_PIPS = 180 # Eve average position when price goes more then 180 pips
XAU_PIP_USD = 0.01 # 1 pip = 0.01 for XAU
QTY_FACTOR_FOR_AVERAGE = 1.65
EVE_MAX_ORDER_COUNT = 20

df[CN['Close']] = pd.to_datetime(df[CN['Close']])
df[CN['Open']] = pd.to_datetime(df[CN['Open']])
df[CN['OpenValue']] = (df['Price'] * 100) * df['Size'] # *100 for USD To Cents
df[CN['BalanceOut']] = df[CN['Profit']].cumsum()
df[CN['BalanceIn']] = df[CN['BalanceOut']] - df[CN['Profit']]

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828 entries, 0 to 827
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Deal            828 non-null    int64         
 1   Open time       828 non-null    datetime64[ns]
 2   Type            828 non-null    object        
 3   Size            828 non-null    float64       
 4   Item            814 non-null    object        
 5   Price           828 non-null    float64       
 6   S/L             828 non-null    int64         
 7   T/P             828 non-null    float64       
 8   Time            828 non-null    datetime64[ns]
 9   Close           828 non-null    float64       
 10  Commission      828 non-null    int64         
 11  Swap            828 non-null    float64       
 12  Profit          828 non-null    float64       
 13  Comment         828 non-null    object        
 14  DK_OPEN_VALUE   828 non-null    float64       
 15  DK_BAL

In [194]:
x= (datetime.date(2023, 6, 17) - datetime.date(2023, 3, 26))
x.days

83

In [195]:
# Alternative Net Recognize

def set_net_id_by_timeline_emulating(df, id_col_name):
    df_res = df.copy()
    df_timeline = pd.DataFrame(columns=['Deal', 'DT', 'Event'])

    # Place deals open and close in timeline order
    for deal, open_time, close in zip(df_res['Deal'], df_res['Open time'], df_res['Time']):
        event = {'Deal': deal, 'DT': open_time, 'Event': 0}
        df_timeline.loc[len(df_timeline)] = event

        event = {'Deal': deal, 'DT': close, 'Event': 1}
        df_timeline.loc[len(df_timeline)] = event

    df_timeline = df_timeline.sort_values(['DT', 'Event'], ascending=True)

    # Emulates opening and closing deal in net
    net_curr = []
    net_full = []
    net_id = None
    for i, row in df_timeline.iterrows():
        if len(net_curr) == 0:
            if net_id is not None:
                # df.loc[df['channel'].isin(['sale','fullprice'])]
                df_res.loc[df_res['Deal'].isin(net_full), [id_col_name]] = net_id
                net_full = []

            net_id = row['Deal']

        if row['Event'] == 0:
            net_curr.append(row['Deal'])
            net_full.append(row['Deal'])
        else:
            net_curr = list(filter(lambda i: i == row['Deal'], net_curr))

    return df_res


# dfan = df[df[CN['Comment']].str.contains(COMMENT_PATTERN_FOR_ORDERS_ONLY, regex=False)]
# dfan = dfan.iloc[430:457]

dfan = set_net_id_by_timeline_emulating(dfan, 'DK_NEW_ID_2')
# dfan



In [ ]:
# Alternative Net Recognize By Start
COMMENT_PATTERN_FOR_START_ORDERS = 'Start'
COMMENT_PATTERN_FOR_SELL_ORDERS = 'SELL'
COMMENT_PATTERN_FOR_BUY_ORDERS = 'BUY'

def set_net_id_by_one_side_start(df: pd.DataFrame, id_col_name):
    df_res = df.copy()

    net_id_sell, net_id_buy = None, None
    net_id_list = []
    for i, row in df_res.iterrows():
        if COMMENT_PATTERN_FOR_START_ORDERS in row[CN['Comment']]:
            if COMMENT_PATTERN_FOR_SELL_ORDERS in row[CN['Comment']]:
                net_id_sell = row[CN['OrderID']]
            if COMMENT_PATTERN_FOR_BUY_ORDERS in row[CN['Comment']]:
                net_id_buy = row[CN['OrderID']]

        if COMMENT_PATTERN_FOR_SELL_ORDERS in row[CN['Comment']]:
            net_id_list.append(net_id_sell) 
        if COMMENT_PATTERN_FOR_BUY_ORDERS in row[CN['Comment']]:
            net_id_list.append(net_id_buy) 
        
    df_res[id_col_name] = net_id_list

    return df_res


# dfan = df[df[CN['Comment']].str.contains(COMMENT_PATTERN_FOR_ORDERS_ONLY, regex=False)]
# dfan = dfan.iloc[430:457]

dfan = set_net_id_by_timeline_emulating(dfan, 'DK_NEW_ID_2')
# dfan



In [196]:
# Set each row NET_ID

# Filter only orders deals without deposits
df_o = df[df[CN['Comment']].str.contains(COMMENT_PATTERN_FOR_ORDERS_ONLY, regex=False)]

df_o = df_o.sort_values(by=[CN['Close'], CN['Open']], ascending=True)

net_id_list = []
qty_list = []
qty_cum = 0
val_list = []
val_cum = 0
close_dt_prev = None
for (order_id, close_dt, qty, val) in zip(df_o[CN['OrderID']], df_o[CN['Close']], df_o[CN['Qty']], df_o[CN['OpenValue']]):
    if close_dt_prev != close_dt:
        net_id_list.append(order_id)
        close_dt_prev = close_dt
        qty_cum = qty
        val_cum = val
    else:
        net_id_list.append(net_id_list[-1])
        qty_cum = qty_cum + qty
        val_cum = val_cum + val

    qty_list.append(qty_cum)
    val_list.append(val_cum)

df_o[CN['NetID']] = net_id_list
df_o[CN['NetID']] = df_o[CN['NetID']].astype(int)
df_o[CN['NetQty']] = qty_list
df_o[CN['NetValue']] = val_list

#todo change loop for shift vector
df_o = df_o.sort_values(by=[CN['NetID'], CN['Open']], ascending=False)
worst_price_list = []
net_id_prev = None
open_price_prev = None
for (net_id, open_price, side) in zip(df_o[CN['NetID']], df_o[CN['OpenPrice']], df_o[CN['Side']]):
    if net_id == net_id_prev:
        worst_price_list.append(open_price_prev)
    else:
        max_price_delta = (OPEN_NEW_ORDER_PRICE_DELTA_PIPS - 1) * XAU_PIP_USD
        if side == 'buy': 
            max_price_delta = -1 * max_price_delta
        worst_price_list.append(open_price + max_price_delta)

    open_price_prev = open_price
    net_id_prev = net_id


df_o[CN['WorstPrice']] = worst_price_list
df_o[CN['Drawdown']] = abs(df_o[CN['NetQty']] * (df_o[CN['WorstPrice']] * 100) - df_o[CN['NetValue']]) # *100 for USD price to Cents
df_o[CN['DrawdownRatio']] = df_o[CN['Drawdown']] / df_o[CN['BalanceIn']]

df_o = df_o.sort_values(by=CN['Open'], ascending=True)


In [197]:
df_o['DK_NET_ID_OLD'] = df_o['DK_NET_ID']
df_o = set_net_id_by_timeline_emulating(df_o, 'DK_NET_ID')
df_o[CN['NetLastPrice']] = df_o['Price']
df_o['NetOpenQty'] = df_o[CN['Qty']]
df_o.loc[df_o[CN['Side']] == 'sell', ['NetIDSell']] = df_o[CN['NetID']]
df_o.loc[df_o[CN['Side']] == 'buy', ['NetIDBuy']] = df_o[CN['NetID']]
df_o

,Deal,Open time,Type,Size,Item,Price,S/L,T/P,Time,Close,...,DK_NET_QTY,DK_NET_VALUE,DK_WORST_PRICE,DK_DRAWDOWN,DK_DRAWDOWN_RATIO,DK_NET_ID_OLD,DK_NET_LAST_PRICE,NetOpenQty,NetIDSell,NetIDBuy
2,69939230,2023-05-26 17:10:33,sell,0.03,XAUUSD,1950.73,0,1949.23,2023-05-26 17:20:48,1949.20,...,0.03,5852.19,1952.52,5.37,0.000026,69939230,1950.73,0.03,69939230.0,NaN
3,69940374,2023-05-26 17:21:00,buy,0.03,XAUUSD,1949.23,0,1950.73,2023-05-26 17:27:21,1950.75,...,0.03,5847.69,1947.44,5.37,0.000026,69940374,1949.23,0.03,NaN,69939230.0
4,69940734,2023-05-26 17:28:00,sell,0.03,XAUUSD,1950.20,0,1948.70,2023-05-26 17:29:24,1948.64,...,0.03,5850.60,1951.99,5.37,0.000026,69940734,1950.20,0.03,69939230.0,NaN
5,69941173,2023-05-26 17:30:00,buy,0.03,XAUUSD,1948.20,0,1944.09,2023-05-26 19:01:11,1944.21,...,0.03,5844.60,1945.79,7.23,0.000035,69941173,1948.20,0.03,NaN,69939230.0
6,69942587,2023-05-26 17:33:00,buy,0.03,XAUUSD,1945.79,0,1944.09,2023-05-26 19:01:11,1944.21,...,0.06,11681.97,1942.41,27.51,0.000133,69941173,1945.79,0.03,NaN,69939230.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
823,72850711,2023-06-16 05:48:02,sell,0.20,XAUUSD,1955.77,0,1955.22,2023-06-16 07:49:01,1955.21,...,0.20,39115.40,1957.59,36.40,0.000028,72850711,1955.77,0.20,72849478.0,NaN
824,72854685,2023-06-16 06:29:01,sell,0.22,XAUUSD,1957.59,0,1955.22,2023-06-16 07:49:01,1955.21,...,0.42,82182.38,1959.38,111.58,0.000086,72850711,1957.59,0.22,72849478.0,NaN
825,72868535,2023-06-16 09:08:01,buy,0.20,XAUUSD,1959.99,0,1961.49,2023-06-16 09:21:38,1961.50,...,0.20,39199.80,1958.20,35.80,0.000028,72868535,1959.99,0.20,NaN,72868535.0
826,72872812,2023-06-16 09:22:01,buy,0.20,XAUUSD,1961.82,0,1962.30,2023-06-16 10:44:25,1962.34,...,0.20,39236.40,1959.88,38.80,0.000030,72872812,1961.82,0.20,NaN,72872812.0


In [198]:
df_o.to_excel('dfo2.xlsx')

In [186]:
# TOP5 nets with biggest drawdown


df_nets = df_o.groupby([CN['NetID']]).agg({
  CN['Open']: 'min',
  CN['Close']: 'max',
  CN['OrderID']: 'count',
  'NetIDSell': 'count',
  'NetIDBuy': 'count',
  CN['Profit']: 'sum',
  'NetOpenQty': 'min',
  CN['Qty']: 'sum',
  CN['OpenPrice']: 'first',
  CN['NetLastPrice']: 'last',
  CN['OpenValue']: 'sum',
  CN['WorstPrice']: 'last',
  CN['BalanceIn']: 'min',
}).reset_index()

df_nets[CN['NetAvgPrice']] = df_nets[CN['OpenValue']] / df_nets[CN['Qty']]
df_nets[CN['Drawdown']] = abs(df_nets[CN['Qty']] * (df_nets[CN['WorstPrice']] * 100) - df_nets['DK_OPEN_VALUE'])  # *100 for USD price to Cents
df_nets[CN['DrawdownRatio']] = df_nets[CN['Drawdown']] / df_nets[CN['BalanceIn']]
df_nets = df_nets.sort_values(by=CN['DrawdownRatio'], ascending=False)

def calc_dd(r):
    kq = QTY_FACTOR_FOR_AVERAGE
    kp = (OPEN_NEW_ORDER_PRICE_DELTA_PIPS - 1) * XAU_PIP_USD
    
    curr_price = r[CN['NetLastPrice']]
    curr_value = r[CN['OpenValue']]
    curr_qty = r[CN['Qty']]
    for i in range(r[CN['OrderID']] + 1, EVE_MAX_ORDER_COUNT + 1):
      curr_price = curr_price + kp
      curr_value = curr_value + (curr_price * 100) * (curr_qty * kq - curr_qty)
      curr_qty = curr_qty * kq

    return abs(curr_value - curr_qty * (curr_price + kp)  * 100)

df_nets['DealOneSide'] = df_nets[['NetIDSell','NetIDBuy']].max(axis=1)
df_nets['DK_DRAWDOWN_20'] = df_nets.apply(calc_dd, axis = 1)
df_nets['DK_DRAWDOWN_20_RATIO'] = df_nets['DK_DRAWDOWN_20'] / df_nets[CN['BalanceIn']]

df_nets['DK_LOT_1000'] = df_nets['NetOpenQty'] / (df_nets[CN['BalanceIn']] / 100 / 1000)

df_nets.head()

,DK_NET_ID,Open time,Time,Deal,NetIDSell,NetIDBuy,Profit,NetOpenQty,Size,Price,...,DK_OPEN_VALUE,DK_WORST_PRICE,DK_BALANCE_IN,DK_NET_AVG_PRICE,DK_DRAWDOWN,DK_DRAWDOWN_RATIO,DealOneSide,DK_DRAWDOWN_20,DK_DRAWDOWN_20_RATIO,DK_LOT_1000
150,72738082,2023-06-15 16:00:01,2023-06-15 17:55:07,18,10,8,22006.96,0.15,131.05,1933.65,...,25617260.66,1960.36,1032138.14,195476.998550,73257.14,0.070976,10,1.758270e+05,0.170352,0.014533
80,71506439,2023-06-07 18:24:00,2023-06-07 20:52:00,11,4,7,11518.93,0.07,56.97,1955.23,...,11077868.40,1939.72,518315.22,194450.911006,27283.56,0.052639,7,2.313751e+06,4.463985,0.013505
14,70200612,2023-05-30 12:24:00,2023-05-30 16:00:20,10,6,4,3405.05,0.03,15.00,1946.55,...,2935196.00,1961.30,205764.92,195679.733333,6754.00,0.032824,6,1.019361e+06,4.954008,0.014580
59,70985775,2023-06-05 03:33:00,2023-06-05 12:41:38,5,1,4,3921.49,0.03,13.56,1946.49,...,2633155.76,1938.11,210795.36,194185.528024,5078.60,0.024093,4,1.126410e+07,53.436166,0.014232
87,71681189,2023-06-08 15:58:00,2023-06-08 20:12:32,7,4,3,11580.56,0.11,37.15,1958.90,...,7307191.73,1970.87,824898.79,196694.259219,14590.32,0.017687,4,1.133929e+07,13.746285,0.013335


# Chapter 1. Account Summary

In [188]:
# Account Summary

df_sum = df.copy()

df_sum['Days'] = df_sum[CN['Open']].dt.date
df_sum['Balance'] = df_sum[CN['Profit']]
df_sum['Deposit'] = 0
df_sum['Withdrawal'] = 0
df_sum['Profit'] = 0
df_sum['IsDealProfit'] = 0
df_sum.loc[df_sum['Comment'].str.contains('Deposit', regex=False), ['Deposit']] = df_sum['Balance']
df_sum.loc[df_sum['Comment'].str.contains('Withdrawal', regex=False), ['Withdrawl']] = df_sum['Balance']
df_sum.loc[df_sum['Comment'].str.contains(COMMENT_PATTERN_FOR_ORDERS_ONLY, regex=False), ['Profit']] = df_sum['Balance']
df_sum.loc[df_sum['Profit'] >= 0, ['IsDealProfit']] = 1
df_sum['AvgDealProfit'] = df_sum['Profit']
df_sum.loc[df_sum['Profit'] >= 0, ['MaxDealProfit']] = df_sum['Profit']
df_sum.loc[df_sum['Profit'] < 0, ['MaxDealLoss']] = df_sum['Profit']

df_sum = df_sum.groupby(lambda x: True).agg({
    'Days': pd.Series.nunique,
    'Balance': 'sum',
    'Deposit': 'sum',
    'Withdrawal': 'sum',
    'Profit': 'sum',
    'Deal': 'count',
    'IsDealProfit': 'sum',
    'AvgDealProfit': 'mean',
    'MaxDealProfit': 'max',
    'MaxDealLoss': 'min',
})

df_sum['Profit %'] = df_sum['Profit'] / df_sum['Balance']
df_sum['ProfitPerDay'] = df_sum['Profit'] / df_sum['Days']
df_sum['ROI'] = df_sum['Profit'] / df_sum['Deposit']
df_sum['WinRate'] = df_sum['IsDealProfit'] / df_sum['Deal']
df_sum['Nets'] = df_o[CN['NetID']].nunique()
df_sum['MinLOT1000'] = df_nets['DK_LOT_1000'].min()
df_sum['AvgLOT1000'] = df_nets['DK_LOT_1000'].mean()
df_sum['MaxLOT1000'] = df_nets['DK_LOT_1000'].max()
df_sum['AvgDealInNet'] = df_nets['Deal'].mean()
df_sum['MaxDealInNet'] = df_nets['Deal'].max()
df_sum['AvgDealOneSideInNet'] = df_nets['DealOneSide'].mean()
df_sum['MaxDealOneSideInNet'] = df_nets['DealOneSide'].max()
df_sum['AvgNetProfit'] = df_nets['Profit'].mean()
df_sum['MaxNetProfit'] = df_nets['Profit'].max()
df_sum['AvgNetDrawdown'] = df_nets['DK_DRAWDOWN'].mean()
df_sum['MaxNetDrawdown'] = df_nets['DK_DRAWDOWN'].max()
df_sum['AvgNetDrawdownRatio'] = df_nets['DK_DRAWDOWN_RATIO'].mean()
df_sum['MaxNetDrawdownRatio'] = df_nets['DK_DRAWDOWN_RATIO'].max()


# df_sum.style.format({
#   'Balance': lambda val: '${val:,.2f}',
#   'Deposit': lambda val: f'${val:,.2f}',
#   'Withdrawal': lambda val: f'${val:,.2f}',
#   'Profit': lambda val: f'${val:,.2f}',
# })

# pd.options.display.float_format = '${:,.2f}'.format

print(f"Торговых дней: {df_sum.iloc[0]['Days']:,.0f}")
print(f"Баланс: ${df_sum.iloc[0]['Balance']/100:,.2f}")
print(f"Пополнений: ${df_sum.iloc[0]['Deposit']/100:,.2f}")
print(f"Снятий: ${df_sum.iloc[0]['Withdrawal']/100:,.2f}")
print('')
print(f"Прибыль: ${df_sum.iloc[0]['Profit']/100:,.2f}")
print(f"Средняя прибыль в день: ${df_sum.iloc[0]['ProfitPerDay']/100:,.2f}")
print('')

roi_d = df_sum.iloc[0]['Deposit']/df_sum.iloc[0]['ProfitPerDay'] if df_sum.iloc[0]['ProfitPerDay'] != 0 else 0
print(f"ROI: {df_sum.iloc[0]['ROI']*100:,.1f}%")
print(f"ROI дней: {roi_d:,.0f}")
print('')
print(f"Ордеров: {df_sum.iloc[0]['Deal']:,.0f}")
print(f"Прибыльных: {df_sum.iloc[0]['IsDealProfit']:,.0f}")
print(f"Win Rate: {df_sum.iloc[0]['WinRate']*100:,.1f}%")
print(f"Средняя прибыль ордера: ${df_sum.iloc[0]['AvgDealProfit']/100:,.2f}")
print(f"Максимальная прибыль ордера: ${df_sum.iloc[0]['MaxDealProfit']/100:,.2f}")
print(f"Максимальный убыток ордера: ${df_sum.iloc[0]['MaxDealLoss']/100:,.2f}")
print('')
print(f"Cеток: {df_sum.iloc[0]['Nets']:,.0f}")
print(f"Лот MIN, AVG, MAX: {df_sum.iloc[0]['MinLOT1000']:,.4f} | {df_sum.iloc[0]['AvgLOT1000']:,.4f} | {df_sum.iloc[0]['MaxLOT1000']:,.4f}")
print(f"Ордеров в сетке в среднем: {df_sum.iloc[0]['AvgDealInNet']:,.1f}")
print(f"Ордеров в сетке максимум: {df_sum.iloc[0]['MaxDealInNet']:,.0f}")
print(f"Ордеров в одну сторону в сетке в среднем: {df_sum.iloc[0]['AvgDealOneSideInNet']:,.1f}")
print(f"Ордеров в одну сторону в сетке максимум: {df_sum.iloc[0]['MaxDealOneSideInNet']:,.0f}")
print(f"Средняя прибыль сетки: ${df_sum.iloc[0]['AvgNetProfit']/100:,.2f}")
print(f"Максимальная прибыль сетки: ${df_sum.iloc[0]['MaxNetProfit']/100:,.2f}")
print(f"Средняя просадка сетки: ${df_sum.iloc[0]['AvgNetDrawdown']/100:,.2f}")
print(f"Максимальная просадка сетки: ${df_sum.iloc[0]['MaxNetDrawdown']/100:,.2f}")
print(f"Средняя просадка сетки от депозита: {df_sum.iloc[0]['AvgNetDrawdownRatio']*100:,.1f}%")
print(f"Максимальная просадка сетки от депозита: {df_sum.iloc[0]['MaxNetDrawdownRatio']*100:,.1f}%")





Торговых дней: 16
Баланс: $13,005.64
Пополнений: $12,394.06
Снятий: $0.00

Прибыль: $611.56
Средняя прибыль в день: $38.22

ROI: 4.9%
ROI дней: 324

Ордеров: 828
Прибыльных: 577
Win Rate: 69.7%
Средняя прибыль ордера: $0.74
Максимальная прибыль ордера: $270.18
Максимальный убыток ордера: $-34.35

Cеток: 156
Лот MIN, AVG, MAX: 0.0120 | 0.0140 | 0.0160
Ордеров в сетке в среднем: 5.2
Ордеров в сетке максимум: 18
Ордеров в одну сторону в сетке в среднем: 3.5
Ордеров в одну сторону в сетке максимум: 10
Средняя прибыль сетки: $3.92
Максимальная прибыль сетки: $220.07
Средняя просадка сетки: $11.12
Максимальная просадка сетки: $732.57
Средняя просадка сетки от депозита: 0.2%
Максимальная просадка сетки от депозита: 7.1%


In [113]:
x = df_o[df_o[CN['NetID']] == 72718771]
# x = df_o
x.head(20)

,Deal,Open time,Type,Size,Item,Price,S/L,T/P,Time,Close,...,Comment,DK_OPEN_VALUE,DK_BALANCE_OUT,DK_BALANCE_IN,DK_NET_ID,DK_NET_QTY,DK_NET_VALUE,DK_WORST_PRICE,DK_DRAWDOWN,DK_DRAWDOWN_RATIO
782,72718771,2023-06-15 15:29:02,sell,0.15,XAUUSD,1926.29,0,1953.22,2023-06-15 17:55:07,1953.22,...,Start SELL[tp],28894.35,1049771.57,1050175.52,72718771,0.15,28894.35,1928.60,34.65,0.000033
783,72722096,2023-06-15 15:31:00,sell,0.17,XAUUSD,1928.60,0,1953.22,2023-06-15 17:55:07,1953.22,...,SELL 2[tp],32786.20,1049353.03,1049771.57,72718771,0.32,61680.55,1930.83,106.01,0.000101
784,72724914,2023-06-15 15:33:01,sell,0.26,XAUUSD,1930.83,0,1953.22,2023-06-15 17:55:07,1953.22,...,SELL 3[tp],50201.58,1048770.89,1049353.03,72718771,0.58,111882.13,1932.63,210.41,0.000201
789,72732143,2023-06-15 15:44:00,sell,0.40,XAUUSD,1932.63,0,1953.22,2023-06-15 17:55:07,1953.22,...,SELL 4[tp],77305.20,1048034.71,1048858.31,72718771,0.98,189187.33,1935.62,503.43,0.000480
792,72741000,2023-06-15 16:01:00,sell,0.62,XAUUSD,1935.62,0,1953.22,2023-06-15 17:55:07,1953.22,...,SELL 5[tp],120008.44,1047010.94,1048102.14,72718771,1.60,309195.77,1937.79,850.63,0.000812
794,72744870,2023-06-15 16:09:01,sell,0.98,XAUUSD,1937.79,0,1953.22,2023-06-15 17:55:07,1953.22,...,SELL 6[tp],189903.42,1045521.30,1047033.44,72718771,2.58,499099.19,1940.02,1425.97,0.001362
797,72753460,2023-06-15 16:26:08,sell,1.58,XAUUSD,1940.02,0,1953.22,2023-06-15 17:55:07,1953.22,...,SELL 7[tp],306523.16,1043483.10,1045568.70,72718771,4.16,805622.35,1942.92,2632.37,0.002518
799,72755691,2023-06-15 16:28:01,sell,2.61,XAUUSD,1942.92,0,1953.22,2023-06-15 17:55:07,1953.22,...,SELL 8[tp],507102.12,1040819.70,1043508.00,72718771,6.77,1312724.47,1945.25,4209.78,0.004034
801,72762225,2023-06-15 16:42:01,sell,4.31,XAUUSD,1945.25,0,1953.22,2023-06-15 17:55:07,1953.22,...,SELL 9[tp],838402.75,1037407.43,1040842.50,72718771,11.08,2151127.22,1948.45,7755.38,0.007451
803,72768254,2023-06-15 16:47:15,sell,7.11,XAUUSD,1948.45,0,1953.22,2023-06-15 17:55:07,1953.22,...,SELL 10[tp],1385347.95,1034038.61,1037430.08,72718771,18.19,3536475.17,1951.58,13448.85,0.012964


In [21]:
# df_o['DK_OPEN_VALUE'] = df_o['Price'] * df_o['Size']
# df_o['DK_CLOSE_VALUE'] = df_o['DK_WORST_PRICE'] * df_o['Size'] - df_o['DK_OPEN_VALUE']
# df_o['DK_LOSS'] = df_o['DK_CLOSE_VALUE'] - df_o['DK_OPEN_VALUE']

x = df_o[df_o[CN['NetID']] == 71535499]
x = df_o[df_o[CN['NetID']] == 72780472]
x .head(20)

,Deal,Open time,Type,Size,Item,Price,S/L,T/P,Time,Close,Commission,Swap,Profit,Comment,DK_BALANCE_OUT,DK_BALANCE_IN,DK_NET_ID,DK_WORST_PRICE,DK_OPEN_VALUE
782,72718771,2023-06-15 15:29:02,sell,0.15,XAUUSD,1926.29,0,1953.22,2023-06-15 17:55:07,1953.22,0,0.0,-403.95,Start SELL[tp],1049771.57,1050175.52,72780472,1928.60,288.9435
783,72722096,2023-06-15 15:31:00,sell,0.17,XAUUSD,1928.60,0,1953.22,2023-06-15 17:55:07,1953.22,0,0.0,-418.54,SELL 2[tp],1049353.03,1049771.57,72780472,1930.83,327.8620
784,72724914,2023-06-15 15:33:01,sell,0.26,XAUUSD,1930.83,0,1953.22,2023-06-15 17:55:07,1953.22,0,0.0,-582.14,SELL 3[tp],1048770.89,1049353.03,72780472,1932.63,502.0158
789,72732143,2023-06-15 15:44:00,sell,0.40,XAUUSD,1932.63,0,1953.22,2023-06-15 17:55:07,1953.22,0,0.0,-823.60,SELL 4[tp],1048034.71,1048858.31,72780472,1935.62,773.0520
792,72741000,2023-06-15 16:01:00,sell,0.62,XAUUSD,1935.62,0,1953.22,2023-06-15 17:55:07,1953.22,0,0.0,-1091.20,SELL 5[tp],1047010.94,1048102.14,72780472,1937.79,1200.0844
794,72744870,2023-06-15 16:09:01,sell,0.98,XAUUSD,1937.79,0,1953.22,2023-06-15 17:55:07,1953.22,0,0.0,-1512.14,SELL 6[tp],1045521.30,1047033.44,72780472,1940.02,1899.0342
797,72753460,2023-06-15 16:26:08,sell,1.58,XAUUSD,1940.02,0,1953.22,2023-06-15 17:55:07,1953.22,0,0.0,-2085.60,SELL 7[tp],1043483.10,1045568.70,72780472,1942.92,3065.2316
799,72755691,2023-06-15 16:28:01,sell,2.61,XAUUSD,1942.92,0,1953.22,2023-06-15 17:55:07,1953.22,0,0.0,-2688.30,SELL 8[tp],1040819.70,1043508.00,72780472,1945.25,5071.0212
801,72762225,2023-06-15 16:42:01,sell,4.31,XAUUSD,1945.25,0,1953.22,2023-06-15 17:55:07,1953.22,0,0.0,-3435.07,SELL 9[tp],1037407.43,1040842.50,72780472,1948.45,8384.0275
803,72768254,2023-06-15 16:47:15,sell,7.11,XAUUSD,1948.45,0,1953.22,2023-06-15 17:55:07,1953.22,0,0.0,-3391.47,SELL 10[tp],1034038.61,1037430.08,72780472,1951.58,13853.4795


In [3]:
import datetime

df_o.groupby([df_o[COLUMN_NAME_OPEN_DATETIME].dt.date]).agg({
  COLUMN_NAME_CLOSE_DATETIME: 'max',
  COLUMN_NAME_NET_ID: pd.Series.nunique,
  COLUMN_NAME_DEAL: 'count',
  'Size': 'sum',
  'DK_OPEN_VALUE': 'sum'
  ''
}).reset_index()

NameError: name 'df_o' is not defined

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(df_o[['Time']])
# plt.scatter(df.index[df.Buy_Signal], df[df.Buy_Signal].Close, marker = '^', color='g')
# plt.scatter(df.index[df.Sell_Signal], df[df.Sell_Signal].Close, marker = 'v', color='r')
# plt.fill_between(df.index, df.Upper, df.Lower, color='grey', alpha=0.3)
# plt.legend(['Close', 'SMA', 'Upper', 'Lower'])
plt.show()